<a href="https://colab.research.google.com/github/Besuf/Llava-colab/blob/main/Llava_Olama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
!ollama pull llava:7b

In [ ]:
!pip install langchain-ollama

In [ ]:
import base64, requests, os
from IPython.display import Markdown, display, Image as IPyImage

def b64_image(path):
    return base64.b64encode(open(path, "rb").read()).decode("utf-8")

def caption_and_tags(image_path, temperature=0.1):
    system = (
        "You are an assistant for image captioning and tag generation. "
        "Reply ONLY as compact JSON: {\"caption\": string, \"tags\": [string,..]}."
    )
    user = "Caption this image in one sentence and generate 8-12 SEO-friendly tags."

    payload = {
        "model": "llava:7b",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user, "images": [b64_image(image_path)]},
        ],
        "stream": False,
        "options": {"temperature": temperature},
    }
    r = requests.post("http://localhost:11434/api/chat", json=payload, timeout=600)
    r.raise_for_status()
    text = r.json().get("message", {}).get("content", "")
    return text

# Example
img = "image1.png"  # upload/set your image path
if not os.path.exists(img):
    raise FileNotFoundError(f"Image not found: {img}. CWD: {os.getcwd()}")

# 1) Show the image
display(IPyImage(filename=img))

out = caption_and_tags(img)
display(Markdown(f"**Caption:** {out['caption']}"))
display(Markdown(f"**Tags:** {', '.join(out['tags'])}"))

